In [1]:
import os
import sys
sys.path.append('..')
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from utils.system import read_ir_from_file


In [3]:
_FLAG_TO_DEVICE_NAME = {
    "Cypress": "AMD Radeon HD 5900",
    "Tahiti": "AMD Tahiti 7970",
    "Fermi": "NVIDIA GTX 480",
    "Kepler": "NVIDIA Tesla K20c",
}

data_path = '../../data/opencl_thread_coarsening'

device_list = ["Cypress", "Tahiti", "Fermi", "Kepler"]


oracle_file = os.path.join(data_path, "pact-2014-oracles.csv")
oracles = pd.read_csv(oracle_file)

runtimes_file = os.path.join(data_path, "pact-2014-runtimes.csv")
df = pd.read_csv(runtimes_file)

In [4]:
oracles

,kernel,cf_Fermi,runtime_Fermi,cf_Kepler,runtime_Kepler,cf_Cypress,runtime_Cypress,cf_Tahiti,runtime_Tahiti
0,binarySearch,1.0,213216.0,16.0,165248.0,2.0,290889.0,4.0,178518.0
1,blackscholes,8.0,1148128.0,4.0,1179008.0,1.0,1783889.0,2.0,962963.0
2,convolution,4.0,376256.0,8.0,543104.0,2.0,515445.0,1.0,105186.0
3,dwtHaar1D,8.0,348064.0,4.0,302304.0,4.0,656222.0,4.0,128000.0
4,fastWalsh,2.0,866464.0,1.0,891872.0,2.0,1432000.0,1.0,640740.0
5,floydWarshall,8.0,114432.0,8.0,122464.0,4.0,167112.0,4.0,95555.0
6,mriQ,2.0,25358464.0,1.0,12119168.0,4.0,34089889.0,1.0,9872592.0
7,mt,4.0,83296.0,2.0,84320.0,4.0,198111.0,16.0,85185.0
8,mtLocal,8.0,75264.0,4.0,77440.0,32.0,171445.0,8.0,75555.0
9,mvCoal,1.0,1037696.0,1.0,795776.0,1.0,2051111.0,1.0,672741.0


In [5]:
class ThreadCoaDataset(Dataset):
    def __init__(self, data_path):
        super().__init__()

        _FLAG_TO_DEVICE_NAME = {
            "Cypress": "AMD Radeon HD 5900",
            "Tahiti": "AMD Tahiti 7970",
            "Fermi": "NVIDIA GTX 480",
            "Kepler": "NVIDIA Tesla K20c",
        }

        device_list = ["Cypress", "Tahiti", "Fermi", "Kepler"]
        oracle_file = os.path.join(data_path, "pact-2014-oracles.csv")
        oracles = pd.read_csv(oracle_file)

        llfiles = pd.read_csv(data_path + "/all.txt", sep="\s+")
        fileNum = llfiles["FileNum"]
        filesname = llfiles["ProgramName"]

        oracles["kernel_path"] = data_path + str("/kernels_ir/") + oracles["kernel"] + str(".ll")

        self.resultant_data = pd.DataFrame()
        for i, platform in enumerate(device_list):
            df_tmp = oracles[['kernel','kernel_path']].copy()
            df_tmp['runtime'] = oracles[f'runtime_{platform}']
            df_tmp['platform'] = _FLAG_TO_DEVICE_NAME[platform]
            df_tmp['cf'] = oracles[f'cf_{platform}']                     
            # df_tmp["device"] = i + 1
            self.resultant_data = pd.concat([self.resultant_data, df_tmp])
        

            
    def __getitem__(self, index):
        kernel = self.resultant_data.iloc[index]["kernel"]
        ir = read_ir_from_file(self.resultant_data.iloc[index]["kernel_path"])
        platform = self.resultant_data.iloc[index]["platform"]
        cf = self.resultant_data.iloc[index]["cf"]
        
        return kernel, ir, platform, cf

    def __len__(self):
        return len(self.resultant_data)
    
    
data_path = '../../data/opencl_thread_coarsening'
dataset = ThreadCoaDataset(data_path)
print(len(dataset))


68


In [6]:
dataset[0]

('binarySearch',
 '; ModuleID = \'binarySearch.cl\'\nsource_filename = "binarySearch.cl"\ntarget datalayout = "e-m:o-i64:64-f80:128-n8:16:32:64-S128"\ntarget triple = "x86_64-apple-macosx10.13.0"\n\n; Function Attrs: nounwind ssp uwtable\ndefine spir_kernel void @A(i32* nocapture, i32, i32, i32) local_unnamed_addr #0 !kernel_arg_addr_space !4 !kernel_arg_access_qual !5 !kernel_arg_type !6 !kernel_arg_base_type !6 !kernel_arg_type_qual !7 {\n  %5 = tail call i64 @_Z13get_global_idj(i32 0) #2\n  %6 = trunc i64 %5 to i32\n  %7 = sub i32 %1, %2\n  %8 = and i32 %7, 31\n  %9 = shl i32 1, %8\n  %10 = shl i32 %9, 1\n  %11 = add i32 %9, -1\n  %12 = and i32 %11, %6\n  %13 = lshr i32 %6, %8\n  %14 = mul i32 %10, %13\n  %15 = add i32 %14, %12\n  %16 = add i32 %15, %9\n  %17 = zext i32 %15 to i64\n  %18 = getelementptr inbounds i32, i32* %0, i64 %17\n  %19 = load i32, i32* %18, align 4, !tbaa !8\n  %20 = zext i32 %16 to i64\n  %21 = getelementptr inbounds i32, i32* %0, i64 %20\n  %22 = load i32, i3